In [ ]:
'''
!pip install "numpy<2.0.0"
'''

In [1]:
import gymnasium as gym
from gymnasium import envs
import numpy as np
from collections import deque
import random
import matplotlib.pyplot as plt
import cv2 
import time
import warnings
import ale_py
import multiprocessing as mp
from datetime import datetime
import os
import pandas as pd
from keras.models import Sequential, clone_model
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from collections import deque
import random
import math

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Please experiment with the numbers, we want to make it to the end, and I know it'll take a while so please do not randomly choose

In [ ]:
# --- Hyperparameters ---
EPISODES = 5000                
BATCH_SIZE = 32                
MEMORY_SIZE = 100000           
GAMMA = 0.99                   
EPSILON_START = 1.0            
EPSILON_MIN = 0.01             
EPSILON_DECAY = 0.995          
LEARNING_RATE = 0.00025        
UPDATE_TARGET_FREQ = 1000      
FRAME_STACK = 4                
SAVE_FREQ = 50                 

# --- Prioritized Experience Replay (PER) ---
class PrioritizedReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)
        self.priorities = deque(maxlen=max_size)
    
    def add(self, state, action, reward, next_state, done):
        max_priority = max(self.priorities) if self.buffer else 1.0
        self.buffer.append((state, action, reward, next_state, done))
        self.priorities.append(max_priority)
    
    def sample(self, batch_size, alpha=0.6):
        priorities = np.array(self.priorities)
        probs = priorities ** alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[i] for i in indices]
        weights = (len(self.buffer) * probs[indices]) ** (-0.4)
        weights /= weights.max()
        
        states = np.array([x[0] for x in samples])
        actions = np.array([x[1] for x in samples])
        rewards = np.array([x[2] for x in samples])
        next_states = np.array([x[3] for x in samples])
        dones = np.array([x[4] for x in samples])
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, indices, errors, offset=0.01):
        for i, error in zip(indices, errors):
            self.priorities[i] = abs(error) + offset

# --- Environment Setup ---
env = gym.make("ALE/Frogger-v5", render_mode="rgb_array")
state_shape = (84, 84, FRAME_STACK)
action_size = env.action_space.n

# --- Model Definition ---
def build_model():
    model = Sequential([
        Conv2D(32, (8, 8), strides=4, activation="relu", input_shape=state_shape),
        Conv2D(64, (4, 4), strides=2, activation="relu"),
        Conv2D(64, (3, 3), strides=1, activation="relu"),
        Flatten(),
        Dense(512, activation="relu"),
        Dense(action_size, activation="linear")
    ])
    model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))
    return model

# --- Frame Stacking ---
class FrameStacker:
    def __init__(self):
        self.frames = deque(maxlen=FRAME_STACK)
    
    def reset(self, state):
        for _ in range(FRAME_STACK):
            self.frames.append(state)
        return np.stack(self.frames, axis=-1)
    
    def append(self, state):
        self.frames.append(state)
        return np.stack(self.frames, axis=-1)

# --- Preprocessing ---
def preprocess_state(state):
    state = np.mean(state, axis=2)  
    state = state[34:194, :]        
    state = state[::2, ::2]         
    state = np.pad(state, ((2,2),(2,2)), mode='constant')
    return state / 255.0

# --- Training Functions ---
def train_dqn():
    model = build_model()
    target_model = clone_model(model)  
    target_model.set_weights(model.get_weights())
    
    memory = PrioritizedReplayBuffer(MEMORY_SIZE)  
    frame_stacker = FrameStacker()
    epsilon = EPSILON_START
    rewards_history = []
    start_time = time.time()
    global_step = 0

    for episode in range(1, EPISODES + 1):
        state, _ = env.reset()
        state = preprocess_state(state)
        state = frame_stacker.reset(state)
        total_reward = 0
        done = False

        while not done:
            global_step += 1

            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(np.expand_dims(state, axis=0), verbose=0)
                action = np.argmax(q_values[0])

            next_state, reward, done, _, _ = env.step(action)
            next_state = preprocess_state(next_state)
            next_state = frame_stacker.append(next_state)
            memory.add(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            if global_step % 4 == 0 and len(memory.buffer) >= BATCH_SIZE:
                states, actions, rewards, next_states, dones, indices, weights = memory.sample(BATCH_SIZE)
                
                next_q_values = target_model.predict(next_states, verbose=0)
                best_actions = np.argmax(model.predict(next_states, verbose=0), axis=1)
                target_q = rewards + GAMMA * next_q_values[np.arange(BATCH_SIZE), best_actions] * (1 - dones)
                
                current_q = model.predict(states, verbose=0)
                td_errors = target_q - current_q[np.arange(BATCH_SIZE), actions]
                memory.update_priorities(indices, td_errors)
                
                target = current_q.copy()
                target[np.arange(BATCH_SIZE), actions] = target_q
                model.fit(states, target, sample_weight=weights, epochs=1, verbose=0)

            if global_step % UPDATE_TARGET_FREQ == 0:
                target_model.set_weights(model.get_weights())

        if epsilon > EPSILON_MIN:
            epsilon *= EPSILON_DECAY

        rewards_history.append(total_reward)

        elapsed_time = (time.time() - start_time) / 60  # in minutes
        remaining_time = (elapsed_time / episode) * (EPISODES - episode)
        print(f"Episode: {episode}/{EPISODES}, Reward: {total_reward}, Epsilon: {epsilon:.2f}, Time Elapsed: {elapsed_time:.2f} mins, Remaining: {remaining_time:.2f} mins")

        if episode % SAVE_FREQ == 0:
            model.save_weights(f"frogger_weights_ep{episode}.weights.h5")
            print(f"Saved weights at episode {episode}")

    plt.plot(rewards_history)
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Frogger DQN Training (PER + Double DQN + Frame Stack)")
    plt.savefig("frogger_training_enhanced.png")
    plt.show()

    return model

# --- Run Training ---
if __name__ == "__main__":
    model = train_dqn()
    model.save_weights("frogger_final_weights_enhanced.weights.h5")

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 1/5000, Reward: 11.0, Epsilon: 0.99, Time Elapsed: 0.35 mins, Remaining: 1739.96 mins
Episode: 2/5000, Reward: 8.0, Epsilon: 0.99, Time Elapsed: 0.69 mins, Remaining: 1716.45 mins
Episode: 3/5000, Reward: 10.0, Epsilon: 0.99, Time Elapsed: 1.03 mins, Remaining: 1717.78 mins
Episode: 4/5000, Reward: 8.0, Epsilon: 0.98, Time Elapsed: 1.32 mins, Remaining: 1648.72 mins
Episode: 5/5000, Reward: 10.0, Epsilon: 0.98, Time Elapsed: 1.71 mins, Remaining: 1710.22 mins
Episode: 6/5000, Reward: 8.0, Epsilon: 0.97, Time Elapsed: 2.05 mins, Remaining: 1707.77 mins
Episode: 7/5000, Reward: 11.0, Epsilon: 0.97, Time Elapsed: 2.41 mins, Remaining: 1720.89 mins
Episode: 8/5000, Reward: 10.0, Epsilon: 0.96, Time Elapsed: 2.72 mins, Remaining: 1695.07 mins
Episode: 9/5000, Reward: 10.0, Epsilon: 0.96, Time Elapsed: 3.08 mins, Remaining: 1710.03 mins
Episode: 10/5000, Reward: 12.0, Epsilon: 0.95, Time Elapsed: 3.43 mins, Remaining: 1712.74 mins
Episode: 11/5000, Reward: 10.0, Epsilon: 0.95, Time 

Episode: 86/5000, Reward: 9.0, Epsilon: 0.65, Time Elapsed: 34.94 mins, Remaining: 1996.58 mins
Episode: 87/5000, Reward: 8.0, Epsilon: 0.65, Time Elapsed: 35.36 mins, Remaining: 1996.67 mins
Episode: 88/5000, Reward: 11.0, Epsilon: 0.64, Time Elapsed: 35.89 mins, Remaining: 2003.29 mins
Episode: 89/5000, Reward: 15.0, Epsilon: 0.64, Time Elapsed: 36.41 mins, Remaining: 2009.28 mins
Episode: 90/5000, Reward: 13.0, Epsilon: 0.64, Time Elapsed: 36.99 mins, Remaining: 2017.83 mins
Episode: 91/5000, Reward: 21.0, Epsilon: 0.63, Time Elapsed: 37.63 mins, Remaining: 2030.02 mins
Episode: 92/5000, Reward: 10.0, Epsilon: 0.63, Time Elapsed: 38.30 mins, Remaining: 2043.08 mins
Episode: 93/5000, Reward: 14.0, Epsilon: 0.63, Time Elapsed: 39.07 mins, Remaining: 2061.49 mins
Episode: 94/5000, Reward: 15.0, Epsilon: 0.62, Time Elapsed: 39.76 mins, Remaining: 2075.15 mins
Episode: 95/5000, Reward: 14.0, Epsilon: 0.62, Time Elapsed: 40.50 mins, Remaining: 2091.32 mins
Episode: 96/5000, Reward: 10.0, 

Episode: 170/5000, Reward: 8.0, Epsilon: 0.43, Time Elapsed: 81.70 mins, Remaining: 2321.10 mins
Episode: 171/5000, Reward: 11.0, Epsilon: 0.42, Time Elapsed: 82.26 mins, Remaining: 2323.05 mins
Episode: 172/5000, Reward: 16.0, Epsilon: 0.42, Time Elapsed: 82.89 mins, Remaining: 2326.60 mins
Episode: 173/5000, Reward: 18.0, Epsilon: 0.42, Time Elapsed: 83.51 mins, Remaining: 2330.07 mins
Episode: 174/5000, Reward: 21.0, Epsilon: 0.42, Time Elapsed: 84.14 mins, Remaining: 2333.71 mins
Episode: 175/5000, Reward: 15.0, Epsilon: 0.42, Time Elapsed: 84.76 mins, Remaining: 2336.88 mins
Episode: 176/5000, Reward: 21.0, Epsilon: 0.41, Time Elapsed: 85.38 mins, Remaining: 2340.32 mins
Episode: 177/5000, Reward: 23.0, Epsilon: 0.41, Time Elapsed: 86.11 mins, Remaining: 2346.36 mins
Episode: 178/5000, Reward: 17.0, Epsilon: 0.41, Time Elapsed: 86.91 mins, Remaining: 2354.42 mins
Episode: 179/5000, Reward: 10.0, Epsilon: 0.41, Time Elapsed: 87.63 mins, Remaining: 2360.20 mins
Episode: 180/5000, Re

Episode: 253/5000, Reward: 24.0, Epsilon: 0.28, Time Elapsed: 140.36 mins, Remaining: 2633.46 mins
Episode: 254/5000, Reward: 18.0, Epsilon: 0.28, Time Elapsed: 141.15 mins, Remaining: 2637.39 mins
Episode: 255/5000, Reward: 20.0, Epsilon: 0.28, Time Elapsed: 141.90 mins, Remaining: 2640.37 mins
Episode: 256/5000, Reward: 22.0, Epsilon: 0.28, Time Elapsed: 142.62 mins, Remaining: 2642.85 mins
Episode: 257/5000, Reward: 14.0, Epsilon: 0.28, Time Elapsed: 143.37 mins, Remaining: 2646.01 mins
Episode: 258/5000, Reward: 22.0, Epsilon: 0.27, Time Elapsed: 144.18 mins, Remaining: 2649.92 mins
Episode: 259/5000, Reward: 18.0, Epsilon: 0.27, Time Elapsed: 144.94 mins, Remaining: 2653.09 mins
Episode: 260/5000, Reward: 21.0, Epsilon: 0.27, Time Elapsed: 145.90 mins, Remaining: 2659.84 mins
Episode: 261/5000, Reward: 10.0, Epsilon: 0.27, Time Elapsed: 146.51 mins, Remaining: 2660.27 mins
Episode: 262/5000, Reward: 24.0, Epsilon: 0.27, Time Elapsed: 147.40 mins, Remaining: 2665.64 mins
Episode: 2

...

Episode: 2377/5000, Reward: 39.0, Epsilon: 0.01, Time Elapsed: 3828.29 mins, Remaining: 4224.49 mins
Episode: 2378/5000, Reward: 32.0, Epsilon: 0.01, Time Elapsed: 3830.25 mins, Remaining: 4223.26 mins
Episode: 2379/5000, Reward: 28.0, Epsilon: 0.01, Time Elapsed: 3832.14 mins, Remaining: 4221.96 mins
Episode: 2380/5000, Reward: 31.0, Epsilon: 0.01, Time Elapsed: 3835.27 mins, Remaining: 4222.02 mins
Episode: 2381/5000, Reward: 37.0, Epsilon: 0.01, Time Elapsed: 3837.52 mins, Remaining: 4221.11 mins
Episode: 2382/5000, Reward: 36.0, Epsilon: 0.01, Time Elapsed: 3839.82 mins, Remaining: 4220.26 mins
Episode: 2383/5000, Reward: 37.0, Epsilon: 0.01, Time Elapsed: 3842.08 mins, Remaining: 4219.35 mins
Episode: 2384/5000, Reward: 29.0, Epsilon: 0.01, Time Elapsed: 3844.87 mins, Remaining: 4219.04 mins
Episode: 2385/5000, Reward: 31.0, Epsilon: 0.01, Time Elapsed: 3846.47 mins, Remaining: 4217.41 mins
Episode: 2386/5000, Reward: 26.0, Epsilon: 0.01, Time Elapsed: 3849.52 mins, Remaining: 421

Episode: 2458/5000, Reward: 36.0, Epsilon: 0.01, Time Elapsed: 4022.14 mins, Remaining: 4159.60 mins
Episode: 2459/5000, Reward: 97.0, Epsilon: 0.01, Time Elapsed: 4024.63 mins, Remaining: 4158.83 mins
Episode: 2460/5000, Reward: 89.0, Epsilon: 0.01, Time Elapsed: 4026.50 mins, Remaining: 4157.44 mins
Episode: 2461/5000, Reward: 35.0, Epsilon: 0.01, Time Elapsed: 4028.70 mins, Remaining: 4156.39 mins
Episode: 2462/5000, Reward: 31.0, Epsilon: 0.01, Time Elapsed: 4031.16 mins, Remaining: 4155.60 mins
Episode: 2463/5000, Reward: 23.0, Epsilon: 0.01, Time Elapsed: 4032.76 mins, Remaining: 4153.92 mins
Episode: 2464/5000, Reward: 33.0, Epsilon: 0.01, Time Elapsed: 4035.59 mins, Remaining: 4153.51 mins
Episode: 2465/5000, Reward: 99.0, Epsilon: 0.01, Time Elapsed: 4039.15 mins, Remaining: 4153.85 mins
Episode: 2466/5000, Reward: 41.0, Epsilon: 0.01, Time Elapsed: 4041.69 mins, Remaining: 4153.14 mins
Episode: 2467/5000, Reward: 106.0, Epsilon: 0.01, Time Elapsed: 4044.58 mins, Remaining: 41

Episode: 2539/5000, Reward: 98.0, Epsilon: 0.01, Time Elapsed: 4224.13 mins, Remaining: 4094.37 mins
Episode: 2540/5000, Reward: 29.0, Epsilon: 0.01, Time Elapsed: 4226.97 mins, Remaining: 4093.84 mins
Episode: 2541/5000, Reward: 36.0, Epsilon: 0.01, Time Elapsed: 4229.35 mins, Remaining: 4092.87 mins
Episode: 2542/5000, Reward: 107.0, Epsilon: 0.01, Time Elapsed: 4232.03 mins, Remaining: 4092.18 mins
Episode: 2543/5000, Reward: 89.0, Epsilon: 0.01, Time Elapsed: 4234.02 mins, Remaining: 4090.84 mins
Episode: 2544/5000, Reward: 19.0, Epsilon: 0.01, Time Elapsed: 4235.61 mins, Remaining: 4089.10 mins
Episode: 2545/5000, Reward: 30.0, Epsilon: 0.01, Time Elapsed: 4238.25 mins, Remaining: 4088.37 mins
Episode: 2546/5000, Reward: 31.0, Epsilon: 0.01, Time Elapsed: 4241.17 mins, Remaining: 4087.92 mins
Episode: 2547/5000, Reward: 26.0, Epsilon: 0.01, Time Elapsed: 4244.00 mins, Remaining: 4087.37 mins
Episode: 2548/5000, Reward: 32.0, Epsilon: 0.01, Time Elapsed: 4247.02 mins, Remaining: 40

Episode: 2620/5000, Reward: 34.0, Epsilon: 0.01, Time Elapsed: 4428.67 mins, Remaining: 4022.99 mins
Episode: 2621/5000, Reward: 41.0, Epsilon: 0.01, Time Elapsed: 4431.03 mins, Remaining: 4021.90 mins
Episode: 2622/5000, Reward: 89.0, Epsilon: 0.01, Time Elapsed: 4433.09 mins, Remaining: 4020.55 mins
Episode: 2623/5000, Reward: 91.0, Epsilon: 0.01, Time Elapsed: 4435.36 mins, Remaining: 4019.39 mins
Episode: 2624/5000, Reward: 97.0, Epsilon: 0.01, Time Elapsed: 4437.58 mins, Remaining: 4018.17 mins
Episode: 2625/5000, Reward: 35.0, Epsilon: 0.01, Time Elapsed: 4439.96 mins, Remaining: 4017.11 mins
Episode: 2626/5000, Reward: 96.0, Epsilon: 0.01, Time Elapsed: 4443.05 mins, Remaining: 4016.68 mins
Episode: 2627/5000, Reward: 82.0, Epsilon: 0.01, Time Elapsed: 4445.10 mins, Remaining: 4015.31 mins
Episode: 2628/5000, Reward: 101.0, Epsilon: 0.01, Time Elapsed: 4447.74 mins, Remaining: 4014.47 mins
Episode: 2629/5000, Reward: 37.0, Epsilon: 0.01, Time Elapsed: 4450.31 mins, Remaining: 40

Episode: 2701/5000, Reward: 34.0, Epsilon: 0.01, Time Elapsed: 4639.81 mins, Remaining: 3949.25 mins
Episode: 2702/5000, Reward: 97.0, Epsilon: 0.01, Time Elapsed: 4642.66 mins, Remaining: 3948.50 mins
Episode: 2703/5000, Reward: 31.0, Epsilon: 0.01, Time Elapsed: 4645.05 mins, Remaining: 3947.35 mins
Episode: 2704/5000, Reward: 25.0, Epsilon: 0.01, Time Elapsed: 4646.97 mins, Remaining: 3945.80 mins
